Using a variational model, creates several outputs for each patient in the validation/test set. Saves into csv. The purpose is to do analyses later on 

In [1]:
import os
cwd = os.getcwd()

# protection against running this cell multiple times
assert os.path.dirname(cwd).split('/')[-1] == 'master-thesis','Oops, directory already changed previously as indended. Ignoring...'

# change working directory (if assert passed)
new_cwd = os.path.dirname(cwd) # parent directory
os.chdir(new_cwd)

# Imports

In [1]:
import os
import json

from rnn_utils import DiagnosesDataset, split_dataset, MYCOLLATE
from rnn_utils import train_one_epoch, eval_model

from mourga_variational.variational_rnn import VariationalRNN

import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torch import nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence, pad_sequence, pack_sequence

import numpy as np
import pandas as pd

from config import Settings; settings = Settings()

# display all outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Parameters

In [2]:
num_passes = 30 # number of (variational) forward passes for each input
model_name = 'golden-oath-84'
dataset_id = 'diag_only'
experiment_id = 'A'

sanity check

In [3]:
model_folder = os.path.join(settings.data_base,settings.models_folder,model_name)
os.path.exists(model_folder)

dataset_folder = os.path.join(settings.data_base,settings.model_ready_dataset_folder,dataset_id)
os.path.exists(dataset_folder)

experiment_folder = os.path.join(settings.data_base,settings.variational_data_folder,experiment_id)
if os.path.exists(experiment_folder):
    raise Exception(f"Experiment {experiment_folder} exists. If you want to overwrite it, manually delete the directory first")
else:
    os.mkdir(experiment_folder)

True

True

# Reproducibility

In [4]:
# Reproducibility
seed = settings.random_seed
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [5]:
batch_size = 64 # really doesn't matter for this notebook since we will only to inference
grouping = 'ccs'

In [6]:
dataset = DiagnosesDataset(os.path.join(dataset_folder,'dataset.json'),grouping)

train_dataset = DiagnosesDataset(os.path.join(dataset_folder,'train_subset.json'),grouping)
val_dataset = DiagnosesDataset(os.path.join(dataset_folder,'val_subset.json'),grouping)
test_dataset = DiagnosesDataset(os.path.join(dataset_folder,'test_subset.json'),grouping)


len(train_dataset)
len(val_dataset)
len(test_dataset)


train_dataloader = DataLoader(train_dataset,batch_size=batch_size,collate_fn=MYCOLLATE(dataset),shuffle=True)
val_dataloader = DataLoader(val_dataset,batch_size=batch_size,collate_fn=MYCOLLATE(dataset)) #batch_size here is arbitrary and doesn't affect total validation speed
test_dataloader = DataLoader(test_dataset,batch_size=batch_size,collate_fn=MYCOLLATE(dataset))

5249

1125

1125

In [7]:
# model hyperparameters path
hypp_save_path = os.path.join(model_folder, 'hyper_parameters.json')
with open(hypp_save_path,'r') as f:
    params_loaded = json.load(f)

# weights path
weights_save_path = os.path.join(model_folder,"weights")

new_model = VariationalRNN(**params_loaded)
new_model.load_state_dict(torch.load(weights_save_path))

<All keys matched successfully>

In [8]:
def variational_forward(model, dataloader, dataset, name, num_passes=2):
    """
    """
    # to make sure the dropout mask is turned on. But we won't train here
    model.train()
    
    # eg:: ccs, icd9, etc..
    code_type = dataset.grouping
    
    int2code = dataset.grouping_data[code_type]['int2code']
    
    assert all(np.diff(list(int2code.keys())) == 1), 'Expecting sorted codes, if this fails then it might be time to update column-naming related code'
    col_names = ['diag_' + str(code) for code in int2code.keys()]
    
    total_loss = 0
    total_seq = 0 #total sequences
    
    sigmoid = nn.Sigmoid()
    
    full_df = None
    
    # variational
    
    for i, batch in enumerate(iter(dataloader)):
        # get the inputs; data is a list of [inputs, labels]
        history_sequences, target_sequences = batch['train_sequences'],batch['target_sequences']
        
        _,lengths = pad_packed_sequence(history_sequences['sequence'])
        relevant_positions = [[i+idx*max(lengths) for i in range(e)] for idx,e in enumerate(lengths)]
        
        # just flatten the list
        relevant_positions = [item for sublist in relevant_positions for item in sublist]

        for pass_ in range(1,num_passes+1):

            outs = model(history_sequences['sequence'])
            
            outs_flattened = outs.view(1,-1,outs.size()[2])
            
            relevant_outs = outs_flattened[:,relevant_positions,:]
            
            relevant_outs = sigmoid(relevant_outs).detach().numpy().squeeze()
            
            df = (pd.DataFrame(relevant_outs,
                               columns=col_names)
                  .assign(pat_id=batch['target_pids'],
                          npass=pass_)
                 )
            
            if full_df is None:
                full_df = df
            else:
                full_df = pd.concat([full_df,df])
    
    full_df['adm_index'] = full_df.groupby(['pat_id','npass']).cumcount()+1
    full_df = full_df.reset_index(drop=True)
    full_df[['pat_id','adm_index','npass']] = full_df[['pat_id','adm_index','npass']].astype(int)
    # reorder columns
    full_df = full_df.set_index(['pat_id','adm_index','npass']).sort_index()
    
    # deterministic
    
    full_df_det = None
    model.eval()
    with torch.no_grad():
         for i, batch in enumerate(iter(dataloader)):
            # get the inputs; data is a list of [inputs, labels]
            history_sequences, target_sequences = batch['train_sequences'],batch['target_sequences']

            _,lengths = pad_packed_sequence(history_sequences['sequence'])
            relevant_positions = [[i+idx*max(lengths) for i in range(e)] for idx,e in enumerate(lengths)]

            # just flatten the list
            relevant_positions = [item for sublist in relevant_positions for item in sublist]
            
            outs = model(history_sequences['sequence'])
            
            outs_flattened = outs.view(1,-1,outs.size()[2])
            
            relevant_outs = outs_flattened[:,relevant_positions,:]
            
            relevant_outs = sigmoid(relevant_outs).detach().numpy().squeeze()
            
            df = (pd.DataFrame(relevant_outs,
                               columns=col_names)
                  .assign(pat_id=batch['target_pids'])
                 )
            
            if full_df_det is None:
                full_df_det = df
            else:
                full_df_det = pd.concat([full_df_det,df])
    
    full_df_det['adm_index'] = full_df_det.groupby(['pat_id']).cumcount()+1
    full_df_det = full_df_det.reset_index(drop=True)
    full_df_det[['pat_id','adm_index']] = full_df_det[['pat_id','adm_index']].astype(int)
    # reorder columns
    full_df_det = full_df_det.set_index(['pat_id','adm_index']).sort_index()
    
    
    
    # Now to store the true labels
    
    full_df_golden = None
    for i, batch in enumerate(iter(dataloader)):
        # get the inputs; data is a list of [inputs, labels]
        history_sequences, target_sequences = batch['train_sequences'],batch['target_sequences']

        _,lengths = pad_packed_sequence(history_sequences['sequence'])
        relevant_positions = [[i+idx*max(lengths) for i in range(e)] for idx,e in enumerate(lengths)]

        # just flatten the list
        relevant_positions = [item for sublist in relevant_positions for item in sublist]

        relevant_labels = target_sequences['sequence'].view(1,-1,target_sequences['sequence'].size()[2])[:,relevant_positions,:].squeeze().detach().numpy()
        
        df = pd.DataFrame(relevant_labels,columns=col_names).assign(pat_id=batch['target_pids'])
        
        if full_df_golden is None:
            full_df_golden = df
        else:
            full_df_golden = pd.concat([full_df_golden,df])
            
    full_df_golden['adm_index'] = full_df_golden.groupby(['pat_id']).cumcount()+1
    full_df_golden = full_df_golden.reset_index(drop=True)
    full_df_golden[['pat_id','adm_index']] = full_df_golden[['pat_id','adm_index']].astype(int)
    # reorder columns
    full_df_golden = full_df_golden.set_index(['pat_id','adm_index']).sort_index()
            
    return full_df,full_df_det,full_df_golden

In [9]:
var,det,golden = variational_forward(new_model,val_dataloader,dataset,'validation',num_passes=num_passes)

# save into file

In [10]:
save_var_path = os.path.join(experiment_folder,'variational_forward.csv')
save_det_path = os.path.join(experiment_folder,'deterministic_forward.csv')
save_golden_path = os.path.join(experiment_folder,'golden.csv')

In [11]:
mapping = {save_var_path:var,
       save_det_path:det,
       save_golden_path:golden
      }
for save_path in mapping:
    mapping[save_path].to_csv(save_path)
    
# also save metadata
metadata = dict(model=model_name,
                num_passes=num_passes,
                dataset_id=dataset_id
               )

metadata_path = os.path.join(experiment_folder,'metadata.json')
with open(metadata_path,'w') as f:
    json.dump(metadata,f)
    
print('Saved!')

Saved!


# test it out

In [12]:
df_var = pd.read_csv(save_var_path,index_col=[0,1,2])

df_var.head(3)
df_var.shape

diag_0    diag_1    diag_2    diag_3    diag_4  \
pat_id adm_index npass                                                     
21     1         1      0.001089  0.213134  0.096240  0.019529  0.000285   
                 2      0.001921  0.303439  0.127245  0.051660  0.001970   
                 3      0.004814  0.294661  0.169196  0.026394  0.000411   

                          diag_5    diag_6    diag_7    diag_8    diag_9  ...  \
pat_id adm_index npass                                                    ...   
21     1         1      0.002353  0.002609  0.002373  0.000421  0.002055  ...   
                 2      0.010256  0.003054  0.001259  0.000144  0.002890  ...   
                 3      0.001923  0.002527  0.001474  0.000524  0.004113  ...   

                        diag_262  diag_263  diag_264  diag_265  diag_266  \
pat_id adm_index npass                                                     
21     1         1      0.000015  0.002675  0.000060  0.001814  0.085567   
                 2      0.000042  0.003102  0.000053  0.001158  0.093928   
                 3      0.000027  0.002275  0.000085  0.003200  0.092658   

                        diag_267  diag_268  diag_269  diag_270  diag_271  
pat_id adm_index npass                                                    
21     1         1      0.048912  0.000341  0.000403  0.000462  0.019775  
                 2      0.037497  0.000279  0.000206  0.000391  0.033320  
                 3      0.069284  0.000547  0.000254  0.000379  0.025546  

[3 rows x 272 columns]

(58290, 272)

In [13]:
df_det = pd.read_csv(save_det_path,index_col=[0,1])

df_det.head(3)
df_det.shape

,,diag_0,diag_1,diag_2,diag_3,diag_4,diag_5,diag_6,diag_7,diag_8,diag_9,...,diag_262,diag_263,diag_264,diag_265,diag_266,diag_267,diag_268,diag_269,diag_270,diag_271
pat_id,adm_index,,,,,,,,,,,,,,,,,,,,,
21,1,0.001347,0.305060,0.154384,0.039799,0.000894,0.005886,0.004791,0.001857,0.000335,0.003764,...,0.000047,0.002931,0.000098,0.003253,0.092871,0.054996,0.000995,0.000600,0.000669,0.030053
23,1,0.002962,0.074933,0.096509,0.014177,0.003340,0.005832,0.017698,0.000444,0.001100,0.004600,...,0.000206,0.001529,0.001064,0.001978,0.161321,0.075045,0.001475,0.000521,0.010034,0.027507
61,1,0.015562,0.245559,0.113891,0.058389,0.012167,0.029827,0.060491,0.011735,0.000562,0.003089,...,0.000596,0.001823,0.000619,0.005601,0.271033,0.277932,0.003031,0.001247,0.006792,0.057917


(1943, 272)

In [14]:
df_golden = pd.read_csv(save_golden_path,index_col=[0,1])

df_golden.head(3)
df_golden.shape

,,diag_0,diag_1,diag_2,diag_3,diag_4,diag_5,diag_6,diag_7,diag_8,diag_9,...,diag_262,diag_263,diag_264,diag_265,diag_266,diag_267,diag_268,diag_269,diag_270,diag_271
pat_id,adm_index,,,,,,,,,,,,,,,,,,,,,
21,1,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
23,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
61,1,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


(1943, 272)